In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [5]:
from expiries import dict_expiries

In [6]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [7]:
bnf_1min = pd.read_csv("../data/nifty_min (2).csv")

In [8]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
777613,nifty,2025-05-30 15:25:00,24741.4000,24742.7000,24740.5000,24741.7000,0
777614,nifty,2025-05-30 15:26:00,24742.2500,24746.2000,24740.3000,24740.3000,0
777615,nifty,2025-05-30 15:27:00,24741.0500,24749.0500,24739.5000,24747.1500,0
777616,nifty,2025-05-30 15:28:00,24746.5500,24746.8000,24731.1000,24745.2500,0
777617,nifty,2025-05-30 15:29:00,24743.6000,24749.3000,24731.8500,24736.6500,0


In [9]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
# bnf_1min = bnf_1min[
#     (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
# ]

In [10]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [11]:
from datetime import date
from bisect import bisect_right

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_right(expiry_dates, input_date.date())
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [12]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8

In [20]:
import pandas as pd
import numpy as np

# def calculate_weekly_levels(df, x_percent):
#     # Convert datetime to Pandas datetime type
#     df['datetime'] = pd.to_datetime(df['datetime'])

#     # Set datetime as index
#     df = df.set_index('datetime')

#     # Define week from Friday to Thursday
#     # df['Week'] = (df.index - pd.DateOffset(days=1)).to_period('W-THU')

#     # Resample to get weekly OHLC (Friday Open - Thursday Close)
#     weekly_df = df.resample('W-FRI').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})

#     # Calculate weekly range and previous week's range
#     weekly_df['Range'] = weekly_df['high'] - weekly_df['low']
#     weekly_df['Prev_Week_Range'] = weekly_df['Range'].shift(1)

#     return weekly_df

def calculate_weekly_levels(df, x_percent):
    # Convert datetime to datetime type and set index
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')

    # Split into two parts: before and after Jan 1, 2025
    cutoff_date = pd.Timestamp('2100-01-01')

    df_before_2025 = df[df.index < cutoff_date]
    df_from_2025 = df[df.index >= cutoff_date]

    # Resample each part with appropriate week rules
    weekly_before_2025 = df_before_2025.resample('W-FRI').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    weekly_from_2025 = df_from_2025.resample('W-TUE').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    # Combine both resampled DataFrames
    weekly_df = pd.concat([weekly_before_2025, weekly_from_2025])
    weekly_df = weekly_df.sort_index()

    # Add weekly range and previous week's range
    weekly_df['Range'] = weekly_df['high'] - weekly_df['low']
    weekly_df['Prev_Week_Range'] = weekly_df['Range'].shift(1)

    return weekly_df.reset_index()

    
async def execute_trading_strategy(df, x_percent, tf, offset, entry_time, re_entry_threshold):
    
    df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'})
    weekly_levels = calculate_weekly_levels(df, x_percent)
    weekly_levels = weekly_levels[:-1]
    weekly_levels['datetime'] = pd.to_datetime(weekly_levels['datetime'])
    weekly_levels = weekly_levels.set_index('datetime')
    print(weekly_levels.to_string())
    tradebook = []
    re_entry_num = 0

    # Iterate over all weeks
    for week_till, row in weekly_levels.iterrows():
        print("Previous Week Till : ", week_till)
        if pd.isna(row['Prev_Week_Range']):  
            continue  # Skip the first week as we need the previous week's range

        # Get current week's Friday open
        current_friday_date = week_till + dt.timedelta(days=1)
        print("Current Friday : ", current_friday_date.date())
        current_friday_open = df.loc[df['datetime'].dt.date >= current_friday_date.date(), 'open'].iloc[0]

        # Dynamic multiplier
        # multiplier = 1 * np.sqrt(row['Range'] / current_friday_open)
        multiplier = x_percent * (row['Range'] / current_friday_open)
        if multiplier*100 > 100*x_percent:            # To avoid any trade with previous week range > 10% of current market open
            continue
        # print(multiplier)
        # Calculate high & low levels based on previous week's range
        high_level = current_friday_open + (x_percent / 100) * row['Range']
        low_level = current_friday_open - (x_percent / 100) * row['Range']

        # high_level = current_friday_open + multiplier * row['Range']      # HL = 23000 + (0.21 * 500) = 23105
        # low_level = current_friday_open - multiplier * row['Range']

        # Round levels to nearest 50
        high_level_strike = np.round(high_level / STRIKE_SPREAD_) * STRIKE_SPREAD_
        low_level_strike = np.round(low_level / STRIKE_SPREAD_) * STRIKE_SPREAD_

        # print(high_level_strike, low_level_strike)

        # Initialize trade activity flags
        pe_trade_active = True
        ce_trade_active = True

        current_week_start = current_friday_date
        current_week_end = current_friday_date + dt.timedelta(days=6, hours=15, minutes=30)
        
        # Filter the DataFrame for the current week's data
        current_week_data = df[(df['datetime'] >= current_week_start) & (df['datetime'] <= current_week_end)]
        # print(current_week_data.to_string())

        expiry = get_next_expiry(current_week_start, index_)
        dte = (expiry - current_week_start.date()).days
        if dte >= 7:
            continue
        ce_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(high_level_strike),
            asset_class='C',
            start_date=current_week_start.date(),
            start_time=dt.time(9, 16),
            end_date=expiry,
            end_time=dt.time(15, 30),
        )
        if not isinstance(ce_df, str) and ce_df is not None:
            # print(ce_df.head())
            temp_df = ce_df.to_pandas()
            ce_entry_price = temp_df['o'].iloc[1]
            # print(temp_df.iloc[1])
            ce_df = resample(ce_df, tf, offset)
            ce_df_pandas = ce_df.to_pandas()
            ce_df_pandas['datetime'] = pd.to_datetime(ce_df_pandas['datetime'])
            # ce_entry_price = ce_df_pandas['o'].iloc[1]
            
        else:
            ce_entry_price = float('nan')

        pe_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(low_level_strike),
            asset_class='P',
            start_date=current_week_start.date(),
            start_time=dt.time(9, 16),
            end_date=expiry,
            end_time=dt.time(15, 30),
        )
        if not isinstance(pe_df, str) and pe_df is not None:
            # print(pe_df.head())
            temp_df = pe_df.to_pandas()
            pe_entry_price = temp_df['o'].iloc[1]
            # print(temp_df.iloc[1])
            pe_df = resample(pe_df, tf, offset)
            pe_df_pandas = pe_df.to_pandas()
            pe_df_pandas['datetime'] = pd.to_datetime(pe_df_pandas['datetime'])
            # pe_entry_price = pe_df_pandas['o'].iloc[1]
        else:
            pe_entry_price = float('nan')
        
        tradebook.append({
            'week_start': current_week_start,
            'entry_time': dt.datetime.combine(current_week_start, entry_time),
            'current week open': current_friday_open,
            'previous week high': row['high'],
            'previous week low': row['low'],
            'previous week range': row['Range'],
            'multiplier%': multiplier*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(high_level_strike),
            'type': 'CE',
            'expiry': expiry,
            "tag": "STRANGLE",
            # 'action': 'SELL'
            'entry price': ce_entry_price,
        })
        tradebook.append({
            'week_start': current_week_start,
            'entry_time': dt.datetime.combine(current_week_start, entry_time),
            'current week open': current_friday_open,
            'previous week high': row['high'],
            'previous week low': row['low'],
            'previous week range': row['Range'],
            'multiplier%': multiplier*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(low_level_strike),
            'type': 'PE',
            'expiry': expiry,
            "tag": "STRANGLE",
            # 'action': 'SELL'
            'entry price': pe_entry_price,
        })
        # print('CE Entry', ce_entry_price, '\nPE Entry', pe_entry_price)

        # Manage trades during the week
        # print(current_week_data.to_string())
        for i in range(1, len(current_week_data)):
            dte = (expiry - (current_week_data['datetime'].iloc[i]).date()).days
            # prev_close = current_week_data['close'].iloc[i - 1]
            curr_close = current_week_data['close'].iloc[i]
            curr_datetime = current_week_data['datetime'].iloc[i]
            # print(curr_datetime, curr_close, high_level, low_level)
            # Square off if breached
            if ce_trade_active and curr_close >= high_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'CE' and 
                        trade['week_start'] == current_week_start and 
                        'exit_time' not in trade
                    ):
                        df_row = ce_df_pandas[ce_df_pandas['datetime'] <= curr_datetime]
                        ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = ce_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('CE Exit', ce_exit_price)
                        break
                ce_trade_active = False  # Mark CE trade as inactive
        
            if pe_trade_active and curr_close <= low_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'PE' and
                        trade['week_start'] == current_week_start and 
                        'exit_time' not in trade
                    ):
                        df_row = pe_df_pandas[pe_df_pandas['datetime'] <= curr_datetime]
                        pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = pe_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('PE Exit', pe_exit_price)
                        break
                pe_trade_active = False  # Mark PE trade as inactive
        
            # Re-enter if closed back inside range
            if not ce_trade_active and curr_close <= high_level and dte >= 0 and re_entry_num < re_entry_threshold:
                df_row = ce_df_pandas[ce_df_pandas['datetime'] >= curr_datetime]
                ce_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('CE Entry', ce_entry_price)
                tradebook.append(
                    {
                        'week_start': current_week_start,
                        'entry_time': curr_datetime,
                        'current week open': current_friday_open,
                        'previous week high': row['high'],
                        'previous week low': row['low'],
                        'previous week range': row['Range'],
                        'multiplier%': multiplier*100,
                        'high level': high_level,
                        'low level': low_level,
                        'strike': int(high_level_strike),
                        'type': 'CE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        # 'action': 'SELL'
                        'entry price': ce_entry_price,
                    }
                )
                ce_trade_active = True  # Reactivate CE trade
                re_entry_num += 1
        
            if not pe_trade_active and curr_close >= low_level and dte >= 0 and re_entry_num < re_entry_threshold:
                df_row = pe_df_pandas[pe_df_pandas['datetime'] >= curr_datetime]
                pe_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('PE Entry', pe_entry_price)
                tradebook.append(
                    {
                        'week_start': current_week_start,
                        'entry_time': curr_datetime,
                        'current week open': current_friday_open,
                        'previous week high': row['high'],
                        'previous week low': row['low'],
                        'previous week range': row['Range'],
                        'multiplier%': multiplier*100,
                        'high level': high_level,
                        'low level': low_level,
                        'strike': int(low_level_strike),
                        'type': 'PE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        # 'action': 'SELL'
                        'entry price': pe_entry_price,
                    }
                )
                pe_trade_active = True  # Reactivate PE trade
                re_entry_num += 1

            if curr_datetime == current_week_data.iloc[-1]['datetime']:
                re_entry_num = 0
                for trade in tradebook:
                    if (
                        trade['week_start'] == current_week_start and
                        'exit_time' not in trade
                    ):
                        trade['exit_time'] = curr_datetime
                        
                        if trade['type'] == 'CE':
                            trade['exit price'] = ce_df_pandas['c'].iloc[-1]
                            # print('CE Exit', ce_df_pandas['c'].iloc[-1])
                        else:
                            trade['exit price'] = pe_df_pandas['c'].iloc[-1]
                            # print('PE Exit', pe_df_pandas['c'].iloc[-1])

    tb = pd.DataFrame(tradebook)
    tb['entry_time'] = pd.to_datetime(tb['entry_time'])
    tb['expiry'] = pd.to_datetime(tb['expiry'])
    tb['dte'] = (tb['expiry'].dt.date - tb['entry_time'].dt.date).apply(lambda x: x.days)
    tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
    tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
    tb['portfolio'] = 20_00_000
    tb['index leverage'] = np.where(tb['type'] == 'PE', 9, 10)
    tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike'] 
    tb['PnL'] = tb['qty'] * tb['final points']
    tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
    tb['Trade Year'] = tb['week_start'].dt.year
    
    return tb


In [21]:
tf = '10m'
offset = '5m'
bnf_1h = resample(pl.DataFrame(bnf_1min), tf, offset)
tb = await execute_trading_strategy(bnf_1h.to_pandas(), 5, tf, offset, dt.time(9, 16), 4)

                 open       high        low      close     Range  Prev_Week_Range
datetime                                                                         
2017-01-06  8210.1000  8306.5000  8134.3000  8248.2000  172.2000              NaN
2017-01-13  8259.1000  8457.6500  8228.0000  8397.8000  229.6500         172.2000
2017-01-20  8390.9500  8460.2000  8341.0000  8343.8500  119.2000         229.6500
2017-01-27  8329.6000  8672.5500  8328.0500  8641.9500  344.5000         119.2000
2017-02-03  8635.4500  8756.9000  8537.8000  8738.5500  219.1000         344.5000
2017-02-10  8785.4500  8821.6500  8715.1500  8794.0500  106.5000         219.1000
2017-02-17  8818.8500  8885.9000  8713.0500  8816.9500  172.8500         106.5000
2017-02-24  8818.5500  8981.9000  8809.8500  8941.2000  172.0500         172.8500
2017-03-03  8943.7000  8992.3500  8860.3000  8902.2000  132.0500         172.0500
2017-03-10  8915.1000  8977.7500  8892.2500  8936.8500   85.5000         132.0500
2017-03-17  9091

In [22]:
# tb.to_csv('nifty_rbws.csv', index=False)

In [23]:
# tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
# tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
# tb['portfolio'] = 1_00_00_000
# tb['index leverage'] = 8
# tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike']
# tb['PnL'] = tb['qty'] * tb['final points']
# tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
# tb['Trade Year'] = tb['week_start'].dt.year

In [24]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [28]:
# tb_may = tb[(tb['Trade Year'] == 2025) & (tb['entry_time'].dt.month == 5)]
# tb_may
tb

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
0,2017-01-21,2017-01-21 09:16:00,8329.6000,8460.2000,8341.0000,119.2000,7.1552,8335.5600,8323.6400,8350,CE,2017-01-25,STRANGLE,40.4000,53.4500,2017-01-23 09:25:00,4,0.9385,-13.9885,2000000,10,2395.2096,-33505.3892,-1.6753,2017
1,2017-01-21,2017-01-21 09:16:00,8329.6000,8460.2000,8341.0000,119.2000,7.1552,8335.5600,8323.6400,8300,PE,2017-01-25,STRANGLE,24.0500,0.0500,2017-01-27 15:25:00,4,0.2410,23.7590,2000000,9,2168.6747,51525.5422,2.5763,2017
2,2017-02-18,2017-02-18 09:16:00,8818.5500,8885.9000,8713.0500,172.8500,9.8004,8827.1925,8809.9075,8850,CE,2017-02-23,STRANGLE,29.4000,38.3000,2017-02-20 09:45:00,5,0.6770,-9.5770,2000000,10,2259.8870,-21642.9379,-1.0821,2017
3,2017-02-18,2017-02-18 09:16:00,8818.5500,8885.9000,8713.0500,172.8500,9.8004,8827.1925,8809.9075,8800,PE,2017-02-23,STRANGLE,39.4500,0.0500,2017-02-23 15:25:00,5,0.3950,39.0050,2000000,9,2045.4545,79782.9545,3.9891,2017
4,2017-02-18,2017-02-20 09:55:00,8818.5500,8885.9000,8713.0500,172.8500,9.8004,8827.1925,8809.9075,8850,CE,2017-02-23,RE-ENTRY,35.5500,35.9500,2017-02-20 10:05:00,3,0.7150,-1.1150,2000000,10,2259.8870,-2519.7740,-0.1260,2017
5,2017-03-25,2017-03-25 09:16:00,9093.4500,9166.9500,9019.4000,147.5500,8.1130,9100.8275,9086.0725,9100,CE,2017-03-30,STRANGLE,37.9000,45.5000,2017-03-28 10:35:00,5,0.8340,-8.4340,2000000,10,2197.8022,-18536.2637,-0.9268,2017
6,2017-03-25,2017-03-25 09:16:00,9093.4500,9166.9500,9019.4000,147.5500,8.1130,9100.8275,9086.0725,9100,PE,2017-03-30,STRANGLE,47.5000,46.9500,2017-03-27 09:25:00,5,0.9445,-0.3945,2000000,9,1978.0220,-780.3297,-0.0390,2017
7,2017-03-25,2017-03-28 09:15:00,9093.4500,9166.9500,9019.4000,147.5500,8.1130,9100.8275,9086.0725,9100,PE,2017-03-30,RE-ENTRY,34.7500,0.0500,2017-03-31 15:25:00,2,0.3480,34.3520,2000000,9,1978.0220,67949.0110,3.3975,2017
8,2017-03-25,2017-03-28 11:25:00,9093.4500,9166.9500,9019.4000,147.5500,8.1130,9100.8275,9086.0725,9100,CE,2017-03-30,RE-ENTRY,40.1500,40.7000,2017-03-28 11:35:00,2,0.8085,-1.3585,2000000,10,2197.8022,-2985.7143,-0.1493,2017
9,2017-03-25,2017-03-28 11:45:00,9093.4500,9166.9500,9019.4000,147.5500,8.1130,9100.8275,9086.0725,9100,CE,2017-03-30,RE-ENTRY,40.7000,41.7000,2017-03-28 11:55:00,2,0.8240,-1.8240,2000000,10,2197.8022,-4008.7912,-0.2004,2017


In [26]:
tb['ROI%'].sum()

283.07647250243116

# SENSEX : 60m , 20% of prev week range

In [27]:
stats = generate_stats(tb, 'RBWS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,44.1356,224,31.6964,2.3940,-0.8225,-13.6267,3.2389,RBWS
2020,90.0317,246,28.4553,4.9742,-1.4668,-34.8500,2.5834,RBWS
2021,33.6617,238,28.9916,3.0886,-1.0618,-17.1122,1.9671,RBWS
2022,69.9255,248,30.6452,3.5176,-1.1478,-17.9647,3.8924,RBWS
2023,-18.5130,258,24.4186,1.9662,-0.7534,-27.4995,-0.6732,RBWS
2024,19.3895,237,31.6456,2.3797,-0.9820,-25.7006,0.7544,RBWS
2025,20.3619,89,33.7079,3.2419,-1.3033,-13.9508,1.4596,RBWS
Overall,258.9928,1540,29.5887,3.0517,-1.0410,-34.8500,7.4316,RBWS


In [45]:
stats = generate_stats(tb, 'RBWS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,31.8420,62,54.8387,1.9814,-1.2688,-9.1495,3.4802,RBWS
2024,112.0380,181,53.0387,2.5874,-1.6042,-10.4319,10.7399,RBWS
2025,-4.1709,72,40.2778,2.4951,-2.7331,-25.2636,-0.1651,RBWS
Overall,139.7091,315,50.4762,2.4410,-1.7618,-25.2636,5.5301,RBWS


In [23]:
tb_ce = tb[tb['type'] == 'CE']
tb_pe = tb[tb['type'] == 'PE']
tb_strangle = tb[tb['tag'] == 'STRANGLE']

In [24]:
stats = generate_stats(tb_strangle, 'STR RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,18.0965,38,44.7368,2.9259,-1.5069,-10.9543,1.6520,STR RBW w Strangle
2024,84.0254,104,52.8846,3.0863,-1.7494,-20.4182,4.1152,STR RBW w Strangle
Overall,102.1219,142,54.3956,2.7167,-1.7683,-20.4182,5.0015,STR RBW w Strangle


In [25]:
tb_ce_str = tb_strangle[tb_strangle['type'] == 'CE']
tb_pe_str = tb_strangle[tb_strangle['type'] == 'PE']

In [26]:
stats = generate_stats(tb_ce_str, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,0.4057,19,36.8421,3.0093,-1.7216,-12.5951,0.0322,RBW w Strangle
2024,47.8503,52,48.0769,3.8190,-1.7639,-9.7507,4.9074,RBW w Strangle
Overall,48.2559,71,50.5495,3.1527,-1.7971,-12.5951,3.8313,RBW w Strangle


In [27]:
stats = generate_stats(tb_pe_str, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,17.6908,19,52.6316,2.8675,-1.2205,-3.9387,4.4915,RBW w Strangle
2024,36.1751,52,57.6923,2.4756,-1.7315,-13.1719,2.7464,RBW w Strangle
Overall,53.8659,71,58.2418,2.3384,-1.7342,-17.6205,3.0570,RBW w Strangle


# TF : 10min , Multiplier : 5 * (prev. week range / current open)
# No trades if previous week range > 10% of current spot price

In [28]:
# tb.to_csv('rbws_sensex.csv', index=False)

In [29]:
roi_summary = tb.groupby('tag').agg(
    total_roi_pct=('ROI%', 'sum'),
    trade_count=('ROI%', 'count'),
    win_count=('ROI%', lambda x: (x > 0).sum())
).reset_index()

# Calculate win rate as a percentage
roi_summary['win_rate_pct'] = (roi_summary['win_count'] / roi_summary['trade_count']) * 100

# Optional: drop win_count column if not needed
roi_summary = roi_summary.drop(columns='win_count')
roi_summary

,tag,total_roi_pct,trade_count,win_rate_pct
0,RE-ENTRY,50.8602,117,58.1197
1,STRANGLE,122.1895,182,54.3956


In [30]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,30.5580,62,53.2258,2.0230,-1.2484,-9.2670,3.2975,RBW w Strangle
2024,111.0611,182,52.1978,2.6144,-1.5783,-10.4319,10.6462,RBW w Strangle
Overall,141.6191,244,55.8528,2.2601,-1.5484,-10.4319,13.5755,RBW w Strangle


In [31]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,30.5580,62,53.2258,2.0230,-1.2484,-9.2670,3.2975,RBW w Strangle
2024,111.0611,182,52.1978,2.6144,-1.5783,-10.4319,10.6462,RBW w Strangle
Overall,141.6191,244,55.8528,2.2601,-1.5484,-10.4319,13.5755,RBW w Strangle


In [32]:
tb.to_csv("RBW w Strangle 10min.csv", index=False)

In [33]:
stats = generate_stats(tb, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,30.5580,62,53.2258,2.0230,-1.2484,-9.2670,3.2975,RBW w Strangle
2024,111.0611,182,52.1978,2.6144,-1.5783,-10.4319,10.6462,RBW w Strangle
Overall,141.6191,244,55.8528,2.2601,-1.5484,-10.4319,13.5755,RBW w Strangle


# Original Stats with 50% Range Multiplier and 1hr TF

In [34]:
weekly_roi = tb.groupby('week_start')['ROI%'].sum().reset_index()
weekly_roi['Trade Year'] = weekly_roi['week_start'].dt.year

In [35]:
weekly_roi

,week_start,ROI%,Trade Year
0,2023-08-26,3.6555,2023
1,2023-09-02,2.6969,2023
2,2023-09-09,1.6382,2023
3,2023-09-16,2.1776,2023
4,2023-09-23,1.6615,2023
5,2023-09-30,1.4335,2023
6,2023-10-07,2.5006,2023
7,2023-10-14,1.4233,2023
8,2023-10-21,2.6436,2023
9,2023-10-28,3.1492,2023


In [36]:
stats = generate_stats(weekly_roi, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2023,30.5580,19,78.9474,2.5494,-1.9209,-6.1460,4.9720,RBW w Strangle
2024,111.0611,52,86.5385,2.8323,-2.3417,-7.1030,15.6359,RBW w Strangle
Overall,141.6191,71,80.2198,2.8624,-1.9949,-7.1030,19.9381,RBW w Strangle


In [37]:
# # SIMULATION
# multiplier_range = [10, 20, 30, 40, 50, 60]
# tf = ['5m', '10m', '15m', '20m', '30m', '45m', '60m']
# # tf = ['10m']
# # multiplier_range = [5]
# # entry_times = [dt.time(13, 30), dt.time(13, 45), dt.time(14, 0), dt.time(14, 20), dt.time(14, 35), dt.time(14, 50)]
# entry_time = dt.time(9, 16)
# for i in multiplier_range:
#     for j in tf:
#         # for entry_time in entry_times:
#         variation = f'Multiplier {i}, TF {j}, Entry Time : {entry_time}'
#         print(variation)
        
#         if j == '10m':
#             offset = '5m'
#         elif j >= '20m':
#             offset = '15m'
#         else:
#             offset = '0m'
            
#         bnf_1h = resample(pl.DataFrame(bnf_1min), j, offset)
#         tb = await execute_trading_strategy(bnf_1h.to_pandas(), i, j, offset, entry_time)
#         tb = tb[tb['Trade Year'] < 2025]
#         if len(tb) > 0:

#             stats = generate_stats(tb, variation)
#             for x, y in stats.items():
#                 z = pd.DataFrame(y)
#                 print(z.to_string())


In [38]:
tb.tail()

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
294,2025-05-24,2025-05-24 09:16:00,81928.9500,82424.1000,80489.9200,1934.1800,47.2160,82315.7860,81542.1140,82300,CE,2025-05-27,STRANGLE,484.0000,574.7500,2025-05-26 09:15:00,3,10.5875,-101.3375,10000000,9,1093.5601,-110818.6513,-1.1082,2025
295,2025-05-24,2025-05-24 09:16:00,81928.9500,82424.1000,80489.9200,1934.1800,47.2160,82315.7860,81542.1140,81500,PE,2025-05-27,STRANGLE,221.0000,396.4000,2025-05-27 09:15:00,3,6.1740,-181.5740,10000000,8,981.5951,-178232.1472,-1.7823,2025
296,2025-05-24,2025-05-26 10:15:00,81928.9500,82424.1000,80489.9200,1934.1800,47.2160,82315.7860,81542.1140,82300,CE,2025-05-27,RE-ENTRY,484.6000,0.0500,2025-05-30 15:15:00,1,4.8465,479.7035,10000000,9,1093.5601,524584.6294,5.2458,2025
297,2025-05-24,2025-05-27 10:15:00,81928.9500,82424.1000,80489.9200,1934.1800,47.2160,82315.7860,81542.1140,81500,PE,2025-05-27,RE-ENTRY,286.0000,208.7500,2025-05-27 12:15:00,0,4.9475,72.3025,10000000,8,981.5951,70971.7791,0.7097,2025
298,2025-05-24,2025-05-27 14:15:00,81928.9500,82424.1000,80489.9200,1934.1800,47.2160,82315.7860,81542.1140,81500,PE,2025-05-27,RE-ENTRY,3.0500,0.0500,2025-05-28 09:15:00,0,0.0310,2.9690,10000000,8,981.5951,2914.3558,0.0291,2025


In [39]:
def round_to_nearest_50(value):
    return 100 * round(value / 100)


async def add_hedges(df):
    # df.drop(columns=['ROI%', 'Trade Year'], inplace=True)
    hedge_pct = 3
    index_name = "SENSEX"
    index_str_for_opt = "sensex"

    # df['Hedge Strike'] = df.apply(lambda row: row['Strike'] + round_to_nearest_100((hedge_pct / 100) * row['Strike']) if 'C' in row['Contract'] else row['Strike'] - round_to_nearest_100((hedge_pct / 100) * row['Strike']), axis=1)
    df["Hedge Strike"] = df.apply(
        lambda row: (
            round_to_nearest_50(row["strike"] * (1 + hedge_pct / 100))
            if row["type"] == "CE"
            else round_to_nearest_50(row["strike"] * (1 - hedge_pct / 100))
        ),
        axis=1,
    )

    # df["Hedge Contract"] = df.apply(
    #     lambda row: get_option_contract_name2(
    #         index_name, row["Hedge Strike"], row["expiry"], row["type"]
    #     ),
    #     axis=1,
    # )

    df["Hedge Entry Price"] = np.nan
    df["Hedge Exit Price"] = np.nan

    df['entry_time'] = pd.to_datetime(df['entry_time'])
    df['exit_time'] = pd.to_datetime(df['exit_time'])

    print(df.head().to_string())

    for i in range(0, len(df)):
        print(df.iloc[i]["week_start"])
        hedge_strike = df.iloc[i]['Hedge Strike']
        hedge_expiry = df.iloc[i]["expiry"]
        hedge_opt_type = df.iloc[i]["type"][-2:-1]
        hedge_entry_datetime = df.iloc[i]["entry_time"] + dt.timedelta(minutes=9)
        hedge_exit_datetime = df.iloc[i]["exit_time"] + dt.timedelta(minutes=10)

        hedge_df = await fetch_data(
            index=index_str_for_opt,
            start_date=hedge_entry_datetime.date(),
            end_date=hedge_exit_datetime.date(),
            start_time=hedge_entry_datetime.time(),
            end_time=hedge_exit_datetime.time(),
            expiry=hedge_expiry,
            strike=int(hedge_strike),
            asset_class=hedge_opt_type,
        )
        if not isinstance(hedge_df, str) and hedge_df is not None:
            hedge_df = hedge_df.to_pandas()
        else:
            print(hedge_df)
            df.loc[i, "Hedge Entry Price"] = 0
            df.loc[i, "Hedge Exit Price"] = 0
            continue
        # print(hedge_df)

        hedge_entry_price = hedge_df.iloc[0]["c"]
        hedge_exit_price = hedge_df.iloc[-1]["c"]

        print(hedge_entry_price, hedge_exit_price)

        df.loc[i, "Hedge Entry Price"] = hedge_entry_price
        df.loc[i, "Hedge Exit Price"] = hedge_exit_price

    return df


tb_with_hedge_tmp = await add_hedges(tb)
# tb_with_hedge

  week_start          entry_time  current week open  previous week high  previous week low  previous week range  multiplier%  high level  low level  strike type     expiry       tag  entry price  exit price           exit_time  dte  slippage  final points  portfolio  index leverage       qty         PnL    ROI%  Trade Year  Hedge Strike  Hedge Entry Price  Hedge Exit Price
0 2023-08-26 2023-08-26 09:16:00         64940.5703          65913.7700         64735.8200            1177.9500      36.2778  65176.1603 64704.9803   65200   CE 2023-09-01  STRANGLE     273.4500    297.8000 2023-08-30 09:15:00    6    5.7125      -30.0625   10000000               9 1380.3681 -41497.3160 -0.4150        2023         67200                NaN               NaN
1 2023-08-26 2023-08-26 09:16:00         64940.5703          65913.7700         64735.8200            1177.9500      36.2778  65176.1603 64704.9803   64700   PE 2023-09-01  STRANGLE     223.3500      0.0500 2023-09-01 15:15:00    6    2.2340      2

In [40]:
tb_with_hedge = tb_with_hedge_tmp

In [41]:
tb_with_hedge["Qty"] = tb_with_hedge["qty"] * 1
tb_with_hedge["Hedge Points"] = (
    tb_with_hedge["Hedge Exit Price"] - tb_with_hedge["Hedge Entry Price"]
)
# tb_with_hedge["PnL"] = tb_with_hedge["Points"] * tb_with_hedge["Qty"]
# tb_with_hedge["PnL w cs"] = tb_with_hedge["Points w cs"] * tb_with_hedge["Qty"]
tb_with_hedge["Hedge PnL"] = tb_with_hedge["Hedge Points"] * tb_with_hedge["Qty"]
tb_with_hedge["Total PnL"] = tb_with_hedge["PnL"] + tb_with_hedge["Hedge PnL"]

In [42]:
tb_with_hedge["Total ROI%"] = tb_with_hedge["Total PnL"] * 100 / 10000000
tb_with_hedge["entry_time"] = pd.to_datetime(tb_with_hedge["entry_time"])
tb_with_hedge["Trade Year"] = tb_with_hedge["entry_time"].dt.year

In [43]:
stats_df8 = pd.DataFrame(
    index=range(2017, 2026),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)
combined_df_sorted = tb_with_hedge
# Iterate over each year
for year in range(2017, 2026):
    # Filter trades for the current year
    year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["Total ROI%"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["Total ROI%"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["Total ROI%"] > 0]["Total ROI%"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["Total ROI%"] < 0]["Total ROI%"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["Total ROI%"].cumsum() - year_trades["Total ROI%"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df8.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df8["Total ROI"].sum()
overall_total_trades = stats_df8["Total Trades"].sum()
overall_win_rate = (combined_df_sorted["Total ROI%"] > 0).mean() * 100
overall_avg_profit = combined_df_sorted[combined_df_sorted["Total ROI%"] > 0][
    "Total ROI%"
].mean()
overall_avg_loss = combined_df_sorted[combined_df_sorted["Total ROI%"] < 0][
    "Total ROI%"
].mean()
overall_max_drawdown = (
    combined_df_sorted["Total ROI%"].cumsum()
    - combined_df_sorted["Total ROI%"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df8.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df8

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN
2023,28.5121,58,53.4483,2.0631,-1.3128,-9.6180,2.9645
2024,81.5940,184,47.2826,2.5234,-1.4221,-9.8947,8.2462
2025,28.6470,57,68.4211,1.5398,-1.7448,-5.9673,4.8006
Overall,138.7531,299.0000,52.5084,2.1882,-1.4422,-9.8947,14.0230


In [44]:
tb.tail(49)

,week_start,entry_time,current week open,previous week high,previous week low,previous week range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit price,exit_time,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year,Hedge Strike,Hedge Entry Price,Hedge Exit Price,Qty,Hedge Points,Hedge PnL,Total PnL,Total ROI%
250,2025-01-18,2025-01-20 13:15:00,76978.5300,77319.5000,76249.7200,1069.7800,27.7942,77192.4860,76764.5740,77200,CE,2025-01-21,RE-ENTRY,294.6500,214.9000,2025-01-21 08:15:00,1,5.0955,74.6545,10000000,9,1165.8031,87032.4482,0.8703,2025,79500,3.7000,2.6000,1165.8031,-1.1000,-1282.3834,85750.0648,0.8575
251,2025-01-18,2025-01-21 09:15:00,76978.5300,77319.5000,76249.7200,1069.7800,27.7942,77192.4860,76764.5740,77200,CE,2025-01-21,RE-ENTRY,188.0500,0.0500,2025-01-24 15:15:00,0,1.8810,186.1190,10000000,9,1165.8031,216978.1088,2.1698,2025,79500,1.7000,0.0500,1165.8031,-1.6500,-1923.5751,215054.5337,2.1505
252,2025-01-18,2025-01-21 11:15:00,76978.5300,77319.5000,76249.7200,1069.7800,27.7942,77192.4860,76764.5740,76800,PE,2025-01-21,RE-ENTRY,398.7000,415.9500,2025-01-21 12:15:00,0,8.1465,-25.3965,10000000,8,1041.6667,-26454.6875,-0.2645,2025,74500,5.2000,5.5500,1041.6667,0.3500,364.5833,-26090.1042,-0.2609
253,2025-01-25,2025-01-25 09:16:00,75700.4300,77337.3600,75641.8700,1695.4900,44.7947,76039.5280,75361.3320,76000,CE,2025-01-28,STRANGLE,285.7000,225.0500,2025-01-28 11:15:00,3,5.1075,55.5425,10000000,9,1184.2105,65774.0132,0.6577,2025,78300,10.0500,1.2000,1184.2105,-8.8500,-10480.2632,55293.7500,0.5529
254,2025-01-25,2025-01-25 09:16:00,75700.4300,77337.3600,75641.8700,1695.4900,44.7947,76039.5280,75361.3320,75400,PE,2025-01-28,STRANGLE,229.2000,0.1000,2025-01-31 15:15:00,3,2.2930,226.8070,10000000,8,1061.0080,240644.0318,2.4064,2025,73100,5.6000,0.0500,1061.0080,-5.5500,-5888.5942,234755.4377,2.3476
255,2025-01-25,2025-01-28 14:15:00,75700.4300,77337.3600,75641.8700,1695.4900,44.7947,76039.5280,75361.3320,76000,CE,2025-01-28,RE-ENTRY,1.2000,0.0500,2025-01-29 08:15:00,0,0.0125,1.1375,10000000,9,1184.2105,1347.0395,0.0135,2025,78300,0.4000,0.0500,1184.2105,-0.3500,-414.4737,932.5658,0.0093
256,2025-02-01,2025-02-01 09:16:00,77637.0100,77605.9600,75267.5900,2338.3700,60.2385,78104.6840,77169.3360,78100,CE,2025-02-04,STRANGLE,413.1000,353.3000,2025-02-04 12:15:00,3,7.6640,52.1360,10000000,9,1152.3688,60079.8976,0.6008,2025,80400,47.5500,0.4500,1152.3688,-47.1000,-54276.5685,5803.3291,0.0580
257,2025-02-01,2025-02-01 09:16:00,77637.0100,77605.9600,75267.5900,2338.3700,60.2385,78104.6840,77169.3360,77200,PE,2025-02-04,STRANGLE,484.6500,378.8000,2025-02-03 09:15:00,3,8.6345,97.2155,10000000,8,1036.2694,100741.4508,1.0074,2025,74900,104.7500,15.3000,1036.2694,-89.4500,-92694.3005,8047.1503,0.0805
258,2025-02-01,2025-02-03 12:15:00,77637.0100,77605.9600,75267.5900,2338.3700,60.2385,78104.6840,77169.3360,77200,PE,2025-02-04,RE-ENTRY,218.8500,0.0500,2025-02-07 15:15:00,1,2.1890,216.6110,10000000,8,1036.2694,224467.3575,2.2447,2025,74900,11.2000,0.0500,1036.2694,-11.1500,-11554.4041,212912.9534,2.1291
259,2025-02-08,2025-02-08 09:16:00,77789.3000,78735.4100,76756.0900,1979.3200,50.8893,78185.1640,77393.4360,78200,CE,2025-02-11,STRANGLE,197.3500,0.0500,2025-02-14 15:15:00,3,1.9740,195.3260,10000000,9,1150.8951,224799.7442,2.2480,2025,80500,4.6500,0.0500,1150.8951,-4.6000,-5294.1176,219505.6266,2.1951


In [45]:
tb_with_hedge.to_csv('rbws_sensex_w_hedge.csv', index=False)